## <b>Scrapping Data Digimon </b>
<hr>

In [24]:
# Import Package
import requests
from bs4 import BeautifulSoup as bs
import xlsxwriter

In [23]:
# Connect to web page
url="http://digidb.io/digimon-list/"
html=requests.get(url)
soup=bs(html.content,"html.parser")

In [25]:
# Prepare for the header 
body=soup.find("tbody")
th= soup.find_all("th")
header=[]
for i in th[::]:
    header.append(i.text)
header.insert(2,"Link")
header[0]=header[0].replace("#","ID")
# print(header)

In [ ]:
# Get data for link image
img=soup.find_all("img")
listImg=[]
for link in img[2:-2]:
    listImg.append(link.get('src'))

In [5]:
# get data beside link image
tr=body.find_all("tr")
digi=[]
for i in tr:
    td=i.find_all("td")
    digi.append([i.text for i in td])

for i in range(len(digi)):
    digi[i].insert(2,listImg[i])
for i in digi:
    i[0]=i[0].replace("\xa0","")
    i[1]=i[1].replace("\xa0","")
    i[1]=i[1].replace(" ","")

In [6]:
#============================================== excel

In [7]:
workbook  = xlsxwriter.Workbook('Digimon.xlsx')
sheet = workbook.add_worksheet()

for i in range(len(header)):
    sheet.write(0,i,header[i])
    
for i in range(len(digi)):
    for j in range(len(digi[i])):
        sheet.write(i+1,j,digi[i][j])
workbook.close()



In [8]:
#=========================================csv

In [9]:
import csv

In [10]:
a=digi
b=header
data=[]
for i in range(len(a)):
    data.append(dict(zip(b,a[i])))
with open("digi.csv","w",newline="") as csvku:
    fd=header
    writer=csv.DictWriter(csvku,fieldnames=fd)
    writer.writeheader()
    writer.writerows(data)

In [11]:
#=======================================json

In [12]:
import json
with open("digi.json","w") as jsonku:
    json.dump(data,jsonku)

In [13]:
#======================================mongodb

In [14]:
import pymongo as pm
urldb="mongodb://localhost:27017"
mongoku = pm.MongoClient(urldb)

db=mongoku["Digimon"]
col=db["Digimon"]
col.insert_many(data)

In [15]:
#======================================mysql

In [16]:
for i in range(len(digi)):
    digi[i]=tuple(digi[i])
head="id,digimon,link,stage,type,attribute,memory,equip,hp,sp,atk,def,intel,spd"
import mysql.connector as mc
db=mc.connect(host="localhost",
          port=3306,
          user="root",
          passwd="password",
          database="digimon")
cs=db.cursor()
# query=("create database digimon")
# query=("create table digimon(id varchar(100),digimon varchar(100),link varchar(100),stage varchar(100),type varchar(100),attribute varchar(100),memory varchar(100),equip varchar(100),hp varchar(100),sp varchar(100),atk varchar(100),def varchar(100),intel varchar(100),spd varchar(100))")
# query="describe digimon"
query=f"insert into digimon values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
# cs.execute(query)
# list(cs)
cs.executemany(query,digi)
db.commit() 
print(cs.rowcount, "Data tersimpan!")



341 Data tersimpan!


<hr>